In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.misc import imresize as imresize
from scipy.misc import imread as imread
from scipy.misc import imsave as imsave
import os
import glob
import pickle
import warnings
from IPython.display import HTML
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip
from scipy.ndimage.measurements import label
warnings.filterwarnings("ignore")
## Import deep learning frameworks
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.models import Model
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.models import Sequential
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input
import keras
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam as adam
from keras.layers import Lambda
from keras.models import model_from_json

/home/tony/anaconda3/lib/python3.6/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '
Using TensorFlow backend.


In [2]:
%matplotlib inline

## Pipeline Functions

In [4]:
# Load trained Model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

# Load model
model = model_from_json(loaded_model_json)

# compile the model
model.compile(optimizer='adam', loss='mse')

# load weights into new model
model.load_weights("model.h5", by_name=True)
print("Loaded model from disk")

Loaded model from disk


In [5]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    
    warnings.simplefilter("ignore")
    
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
            
    # Return the list of windows
    return window_list

In [35]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    
    for window in windows:
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        test_img = np.array(test_img)
        test_img = np.reshape(test_img, (1,64,64,3))

        prediction = model.predict(test_img, batch_size=1)

        if prediction >= 0.9:
            # Append window position to list
            on_windows.append(window)

    #8) Return windows for positive detections
    return on_windows

In [28]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [41]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

In [42]:
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    winbox = []
    
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        
        if bbox[0][0] > 800:
            # Draw the box on the image
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
            
    # Return the image
    return img

In [43]:
def final_output(image, hot_windows):
    box_list = hot_windows
    
    # Read in image similar to one shown above 
    heat  = np.zeros_like(image[:,:,0]).astype(np.float)

    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    heat1 = apply_threshold(heat,0.9)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat1, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    
    return draw_img, heatmap

## Main Pipeline

### Load Video and Extract frames

In [46]:
# Load Video
clip = VideoFileClip("project_video.mp4")

# Get the frames from the video
frames = []
for x in clip.iter_frames():
    frames.append(x)

# Convert the sequence to numpy array
frames = np.array(frames)

print('Frame Sequence shape: ', frames.shape)

Frame Sequence shape:  (1260, 720, 1280, 3)


### Pipeline

In [49]:
# To save
detection_map = []

print('Starting: ', end=" ")

# From First frame to the end
for i in range(1, frames.shape[0]):
    
    # Get frame
    frame = frames[i, :, :, :]
    
    # Create two variables image and draw_image
    image = frame
    draw_image = np.copy(frame)
    
    # Sliding window of different sizes.
    windows1 = slide_window(image, x_start_stop=[None, None], y_start_stop=[380, 580], 
                    xy_window=(96, 96), xy_overlap=(0.5, 0.5))
    windows2 = slide_window(image, x_start_stop=[None, None], y_start_stop=[480, 680], 
                        xy_window=(128, 128), xy_overlap=(0.5, 0.5))
    windows3 = slide_window(image, x_start_stop=[None, None], y_start_stop=[480, 720], 
                        xy_window=(256, 256), xy_overlap=(0.5, 0.5))
    
    windows = []

    windows.extend(windows1)
    windows.extend(windows2)
    windows.extend(windows3)

    # Pass the resulting windows for prediction
    hot_windows = search_windows(image, windows)
    
    if hot_windows != []:
        # Pass the resulting predictions to avoid duplicates and false positives
        draw_img_final, heatmap = final_output(draw_image, hot_windows)
        detection_map.append(draw_img_final)
                
    else:
        # Append the final result
        detection_map.append(frame)
    
    # Progress Bar, every 100 frames.
    if i%100 == 0:
        print ('>', end= " ")

print ('end', end=" ")

Starting:  > > > > > > > > > > > > end 

### Save the output to output_images directory

In [50]:
# Save the output image
print('Saving: >', end=" ")

for j in range(len(detection_map)):
    # Get the output image
    image = detection_map[j]
    
    # Filename to save
    name  = 'output_images/'+(str(j)+'.jpg')
    
    # Save the image to directory
    imsave(name, image)
    
    # Increment counter
    j += 1
    
    # Progress Bar, every 100 frames
    
    if j%100 == 0:
        print ('>', end= " ")

print ('end', end=" ")

Saving: > > > > > > > > > > > > > end 

### Convert Output Frames to Video

In [51]:
# Define the codec and create VideoWriter object

import skvideo.io

writer = skvideo.io.FFmpegWriter("output.mp4", outputdict={'-vcodec': 'libx264', '-b': '30000000'}) 

for i in range(1259):
    filename = 'output_images/' + str(i) + '.jpg'
    read = imread(filename, False, 'RGB')
    read = np.array(read)
    
    # Write to video
    writer.writeFrame(read)

writer.close()

In [4]:
HTML("""
<video width="320" height="200" controls>
  <source src="{0}">
</video>
""".format("output.mp4"))